#### Run GeoTopic Parser End-to-End
1. Kill all java processes and kill tika server is already running (refer to Errors section fo ReadMe)
    - `killall java`
    - List current processes: `jps`
        - Kill all tika processes with: `kill <process number>`
2. Navigate to directory /3_Tika_GeoTopic_Parser and run command to start lucene server: `lucene-geo-gazetteer -server`
    - Should see this if working: `INFO: Starting ProtocolHandler ["http-nio-8765"]`
    - If its not working, re-run command to set up path inside of /lucene-geo-gazetteer/src/main/bin: `export PATH=$PWD:$PATH`
3. In new terminal window, navigate to directory /3_Tika_GeoTopic_Parser and run command to start geotopic server: `./geotopic-server`
    - Should see this if working: `INFO  [main] 16:25:04,222 org.apache.tika.server.core.TikaServerProcess Started Apache Tika server ff835cb6-9aa1-4817-ba8d-d035eb174c87 at http://localhost:9998/`
4. In new terminal window again, navigate to directory /3_Tika_GeoTopic_Parser and run command to test servers: `curl -T polar.geot -H "Content-Type: application/geotopic; filename=polar.geot" http://localhost:9998/rmeta | python -mjson.tool`
    - Should Get if running correctly: `United States, 39.76, -98.5`

In [1]:
from tika import parser
import os
import pandas as pd

In [2]:
# df_master_dataset = pd.read_csv('../Datasets/Master_Dataset.tsv', delimiter='\t')
df_master_dataset = pd.read_csv('../Datasets/Master_Dataset_Detoxified_Complete.csv')

/var/folders/fn/pdhscw6s1wz7r1wk9ssprtz80000gn/T/ipykernel_16510/2506762431.py:2: DtypeWarning: Columns (0,1,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_master_dataset = pd.read_csv('../Datasets/Master_Dataset_Detoxified_Complete.csv')


In [3]:
df_master_dataset['GeoTopic Name'] = pd.Series(dtype=str)
df_master_dataset['GeoTopic Latitude'] = pd.Series(dtype=float)
df_master_dataset['GeoTopic Longitude'] = pd.Series(dtype=float)

In [4]:
directory_path = 'Pixstory_Posts_Text_Files/'

directory = os.listdir(directory_path)

In [5]:
# Takes about 25 mins to run on Macbook Air 2020 M1 Chip
cnt = 0

for file in directory:

    cnt +=1

    # check if the current file is a file (not a subdirectory)
    # if os.path.isfile(os.path.join(directory_path_v2, file)):

    # split the string by '_' character and get the second element (index 1)
    index = int(file.split('_')[1])
    # print(index)

    # Parse the file with Tika and the GeoTopic Parser
    parsed = parser.from_file(directory_path + file, headers={'Content-Type' : 'application/geotopic'})

    try:
        name = parsed['metadata']['Geographic_NAME']
        latitude = parsed['metadata']['Geographic_LATITUDE']
        longitude = parsed['metadata']['Geographic_LONGITUDE']

        df_master_dataset.loc[index, 'GeoTopic Name'] = name
        df_master_dataset.loc[index, 'GeoTopic Latitude'] = latitude
        df_master_dataset.loc[index, 'GeoTopic Longitude'] = longitude

    except:
        df_master_dataset.loc[index, 'GeoTopic Name'] = "NaN"
        df_master_dataset.loc[index, 'GeoTopic Latitude'] = "NaN"
        df_master_dataset.loc[index, 'GeoTopic Longitude'] = "NaN"

    finally:
        # print(cnt)
        try: 
            print(f"Processed file {cnt}: {file} - {name, latitude, longitude}", end='\r')
        except:
            print(f"Processed file {cnt}: {file}", end='\r')
        finally:
            continue

In [ ]:
df_master_dataset

,Unnamed: 0,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Date (No Timestamp),Interest,Narrative LangDetect,Narrative TikaDetect,Clean Narrative,translated_text,GeoTopic Name,GeoTopic Latitude,GeoTopic Longitude
0,0,121169,STY1659426957,103,USR1606807023,female,34,Trend of the Year: Barbiecore,"The colour of the year is here, and it's *drum...",https://image.pixstory.com/Pixstory-image-1659...,1/12/20 12:47,1/12/20,"trends, fashion, barbie",[en:0.9999974986518982],en,The colour of the year is here and it s drumro...,The colour of the year is here and it s drumro...,Barbie,35.0065,-6.07477
1,1,127727,STY1660634861,103,USR1606807023,female,34,Abomination of the day,We Indians do love to bastardise our foods- Ch...,https://image.pixstory.com/Pixstory-image-1660...,1/12/20 12:47,1/12/20,"Food, momos, weird menus",[en:0.9999957708537239],en,We Indians do love to bastardise our foods Chi...,We Indians do love to bastardise our foods Chi...,NaN,NaN,NaN
2,2,123665,STY1660027898,103,USR1606807023,female,34,Shameful headline in 2022,Can professors not have personal lives? \n\nAd...,https://image.pixstory.com/Pixstory-image-1660...,1/12/20 12:47,1/12/20,"misogyny, st xaviers",[en:0.999996152302975],en,Can professors not have personal lives Additio...,Can professors not have personal lives Additio...,NaN,NaN,NaN
3,3,130517,STY1661151635,103,USR1606807023,female,34,Woman lawyer arrested for abusing security guard,"She was recorded on video manhandling him, sho...",https://image.pixstory.com/Pixstory-image-1661...,1/12/20 12:47,1/12/20,"Technology, History, Food, Entertainment, Spor...",[en:0.9999974174347146],en,She was recorded on video manhandling him shou...,She was recorded on video manhandling him shou...,NaN,NaN,NaN
4,4,125466,STY1660372361,109,USR1606851217,others,31,What is the Inflation Reduction Act?,The House passed the Inflation Reduction Act o...,https://image.pixstory.com/Pixstory-image-1660...,2/12/20 1:03,2/12/20,us government,[en:0.9999966874383718],en,The House passed the Inflation Reduction Act o...,The House passed the Inflation Reduction Act o...,United States,39.76,-98.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,125349,STY1660325841,71665,USR1660325291,NaN,18,COLLEGE LIFE,MODR CLASS,https://image.pixstory.com/Pixstory-image-1660...,12/8/22 22:58,12/8/22,"college life, college life umang",[en:0.9999935969925867],de,MODR CLASS,MODR CLASS,NaN,NaN,NaN
94996,94996,125345,STY1660325638,71673,USR1660325486,NaN,17,The one with the bestie😍,College life without a friend is useless. Enjo...,https://image.pixstory.com/Pixstory-image-1660...,12/8/22 23:01,12/8/22,college life,[en:0.9999973276572175],en,College life without a friend is useless Enjoy...,College life without a friend is useless Enjoy...,NaN,NaN,NaN
94997,94997,125355,STY1660326971,71697,USR1660326022,NaN,19,College,"Sheridan college, HazelMcCallion Canpus",https://image.pixstory.com/Pixstory-image-1660...,12/8/22 23:10,12/8/22,"college life, college life umang","[en:0.5714278464445594, it:0.4285718905110678]",it,Sheridan college HazelMcCallion Canpus,Sheridan college HazelMcCallion Canpus,NaN,NaN,NaN
94998,94998,125351,STY1660326276,71698,USR1660326026,NaN,21,A tale of friendship,Check this out.,https://image.pixstory.com/Pixstory-image-1660...,12/8/22 23:10,12/8/22,friends,[en:0.9999967202991251],en,Check this out,Check this out,NaN,NaN,NaN


In [ ]:
df_master_dataset.to_csv('../Datasets/Master_Dataset_TranslactionsDetoxifiedGeoTopic.csv', index=False)